In [1]:
import sys
sys.path.append("..")
import os

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import torchvision.models as models
import torchvision.transforms as transforms
from vasync.models.pretext_model import ClassifierPretextModel
from vasync.utils.common import load_config
from PIL import  Image

In [3]:
def load_model(config_path, checkpoint_path):
    config = load_config(config_path)
    pretext_base_model = config["pretext_base_model"]
    num_cls = config["num_cls"]
    model = ClassifierPretextModel(pretext_base_model=pretext_base_model, 
                                   num_cls=num_cls)
    state_dict = torch.load(checkpoint_path)
    model.load_state_dict(state_dict)
    return model

In [24]:
checkpoint_path = "../outputs/img_cls/img_cls_pseudolbl30/checkpoint_epoch_127.pt"
config_path = "../experiments/img_cls_pseudolbl_30/config.yml"
model = load_model(config_path, checkpoint_path)


In [25]:
feature_extractor = nn.Sequential(*list(model.children())[:-1])

In [26]:
transform = transforms.Compose([transforms.Resize(250),
                                transforms.RandomCrop(245),
                                transforms.ToTensor()])

In [27]:
image_path = "image.jpg"
img = Image.open(image_path)
img = transform(img)
# img = torch.randn(3, 245, 245)

In [28]:
out_cls = model(img.unsqueeze(0))
out_feat = feature_extractor(img.unsqueeze(0))
softmax = nn.Softmax()

In [29]:
cls = torch.argmax(softmax(out_cls.squeeze(0)))
print(cls)

tensor(8, grad_fn=<NotImplemented>)


/home/hhemati/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [30]:
print(out_feat.view(-1))
print(torch.mean(out_feat))
print(torch.std(out_feat))

tensor([0.7491, 0.7407, 0.6550,  ..., 0.7606, 0.7630, 0.6945],
       grad_fn=<ViewBackward>)
tensor(0.7384, grad_fn=<MeanBackward0>)
tensor(0.0701, grad_fn=<StdBackward0>)
